In [1]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import json

import unicodedata
import re

In [2]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [3]:
start_at = 1
end_at = 13

In [4]:
client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
  )

In [5]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [6]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [7]:
chinese_extraction_prompt = "The provided content is a Chinese translation of the Ainu original text in Markdown. Please extract the Chinese translation from the last output. Only extract the main text."
english_extraction_prompt = "The provided content is an English translation of the Ainu original text in Markdown. Please extract the English translation from the last output. Only extract the main text."

In [8]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [9]:
# process the translation in batch mode

for song_no in range(start_at, end_at + 1):

    md_name_part = get_output_file_name_key(ainu_titles[song_no - 1])

    with open(f"Chiri_Japanese_Translation/story_translation_{song_no}.txt", "r", encoding="utf8") as f:
        japanese_story = f.read()
        japanese_story = unicodedata.normalize('NFKC', japanese_story)

    with open(f"LLM_prompts_and_raw_translations_main_text/Chinese_Translation/{song_no}_{md_name_part}_to_Chinese.md", "r", encoding="utf8") as f:
        md_chinese_text = f.read()
        md_chinese_text = unicodedata.normalize('NFKC', md_chinese_text)

        chinese_translation_extraction = generate(client,generate_content_config,model,input_text=md_chinese_text,prompt=chinese_extraction_prompt)

        chinese_translation_extraction = re.sub(r'```','',chinese_translation_extraction)

    with open(f"LLM_prompts_and_raw_translations_main_text/English_Translation/{song_no}_{md_name_part}_to_English.md", "r", encoding="utf8") as f:
        md_english_text = f.read()
        md_english_text = unicodedata.normalize('NFKC', md_english_text)

        english_translation_extraction = generate(client,generate_content_config,model,input_text=md_english_text,prompt=english_extraction_prompt)
        english_translation_extraction = re.sub(r'```','',english_translation_extraction)


    data_dict = {'ainu_title': ainu_titles[song_no - 1], 'japanese_title': japanese_titles[song_no], 'japanese_translation': japanese_story, 'chinese_translation': chinese_translation_extraction, 'english_translation': english_translation_extraction}

    with open(f"raw_translations_main_text_json/{song_no}_{md_name_part}.json", "w", encoding="utf8") as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)



貓頭鷹神親自吟唱的歌謠
「銀之滴 簌簌落下 周圍」

「銀之滴 簌簌落下 周圍,金之滴
簌簌落下 周圍。」我一邊唱著這首歌,
一邊沿著河流往下,經過人類的村莊,
往下看時,
發現以前的窮人現在變成了有錢人,以前的有錢人
現在卻變成了窮人。
海邊,人類的孩子們拿著玩具小弓和
玩具小箭在玩耍。
「銀之滴 簌簌落下 周圍
金之滴 簌簌落下 周圍。」一邊唱著這首歌,
一邊經過孩子們的上方,(孩子們)一邊在我的下方奔跑,
一邊說道:
「美麗的鳥! 神明的鳥!
來吧,射箭射那隻鳥,
射中神明的鳥的人,第一個拿到的人
才是真正的勇者,真正的強者!」
一邊說著,以前是窮人現在變成有錢人的
孩子們,用金製小弓和金製小箭
瞄準我射箭,我讓金製小箭
從下方穿過,或從上方穿過。
在這些孩子之中,
有一個孩子拿著普通的(木製)小弓和普通的小箭
加入夥伴們。我看到他,
像是窮人家的孩子,從他的穿著
就能看出來。但是仔細看他的眼神,
卻像是偉人的後代,獨自一人與眾不同地
加入夥伴們。他也用普通的小弓和
普通的小箭瞄準我,
以前是窮人現在變成有錢人的孩子們就大笑著
說道:
「真是可笑啊,窮人家的孩子,
那隻鳥,神明的鳥,連我們的
金製小箭都不會理睬,像你這樣的
窮人家的普通箭、腐爛木頭做的箭,
那隻鳥,神明的鳥,怎麼可能會
理睬呢。」
說著,就踢打、毆打那個貧窮的孩子。但是貧窮的孩子
一點也不在意,只是瞄準著我。
我看到那個樣子,覺得非常可憐。
「銀之滴 簌簌落下 周圍,
金之滴 簌簌落下 周圍。」一邊唱著這首歌,
一邊緩慢地在大空中
畫著圓圈。貧窮的孩子
一隻腳遠遠地站立,一隻腳靠近地站立,
緊緊咬著下唇,瞄準著,
咻地一聲射了出去。小小的箭美麗地飛著,
朝我飛來,於是我伸出手
接住了那支小箭。
我一邊旋轉,一邊
劃破風聲,舞動著降落。
於是,那些孩子們奔跑著,
揚起沙塵暴,互相競爭。
當我落在地上時,幾乎同時,第一個
貧窮的孩子衝過來,拿到了我。
於是,以前是窮人現在變成有錢人的
孩子們從後面跑過來,
說了二十句、三十句惡毒的話,
推打、毆打貧窮的孩子,
「可惡的孩子,窮人家的孩子,
我們想先做的事情,你竟然搶先一步。」
說著,貧窮的孩子就覆蓋在我的身上,
用自己的肚子緊緊地壓住我。
掙扎著、掙扎著,好不容易從人群的縫隙中
跳了出來,然後,就飛快地跑了起來。
以前是窮人現在變成有錢人的孩子們
